# Note to Levi

* I decided that any NRMSE above 1 should probably be considered a 'failure', since its worse than just predicting the mean anyways.
______________
* I made a method run_test() (under LCESN experiments) which takes a dictionary and filename (to save results) as input and runs tests using the parameters specified in the beginning of the function.
    * right now it's written for LCESN, but can be easily changed so that the specific ESN class is passed as a parameter (although we'll have to change the 
    * For now you have to hard code the params you want in the beginning of the function, but I'll change it soon so you can pass params for grid searching etc.
    * The results dictionary keys should contain all the necessary param information. The keys are created in a special way to make show_nrmse_histograms() work correctly.   
______________
* I also created a function show_nrmse_histograms() which takes one of the results dicts from run_test() and shows some cool histograms of generative NRMSEs yielded using the param settings in the dict. 
    * There's an example down below under 'NRMSE histograms' section
    
_____________
    
We should be able to use these methods to get most or all of our comparable results between modern ESN types and param settings. If you could use these to run your experiments too that'd be great (although they need to be modified before other types of models can be tried).

I'll generalize the methods within a few days so you can run your experiments on them too (although if you want to do some tests before then you can do it if you wabt)

# Imports and loading data

In [1]:
import numpy as np
import os, time
import matplotlib.pyplot as plt
import pickle as pkl
from ESN.ESN import ESN, LCESN, EESN, DHESN
from MackeyGlass.MackeyGlassGenerator import run
from Helper.utils import nrmse
from ipywidgets import IntProgress
from IPython.display import display

data = np.array(run(21100)).reshape(-1, 1)
split = 20000
X_train = data[:split-1]
y_train = data[1:split]
valid_data = data[split:].squeeze()

data_mean = np.mean(data.squeeze())

# zero the data (for PCA)
X_train -= data_mean
y_train -= data_mean
valid_data -= data_mean

DONE


# Basic ESN experiments

# LCESN experiments

In [2]:
all_results = dict()

In [3]:
def run_test(esn_class, all_results, fname=None):
    assert esn_class in [ESN, LCESN, EESN, DHESN]
    
    echo_params = 0.85
    regulariser = 1e-3
    num_reservoirs = 5
    reservoir_sizes = [int(np.ceil(1000. / num_reservoirs))]*num_reservoirs
    in_weights = {'strategies': 'binary', 'scales': 0.2, 'offsets': 0.5}
    res_weights = {'strategies': 'uniform', 'spectral_scales': 1., 'offsets': 0.5}
    n_runs = 50

    assert sum(reservoir_sizes) == 1000
    
    in_notebook = os.environ['_'][-7:] == 'jupyter'
    
    if in_notebook:
        progress_bar = IntProgress(value=0, min=0, max=n_runs)
        display(progress_bar)
        
    results = []
    start_time = time.time()
    for run_num in range(n_runs):
        if not in_notebook:
            print('Run %d' % (run_num+1))
        # create and train model
        lcesn = esn_class(1, 1, num_reservoirs, reservoir_sizes, echo_params, 
                      regulariser=regulariser)
        lcesn.initialize_input_weights(
            strategies=in_weights['strategies'], scales=in_weights['scales'],
            offsets=in_weights['offsets']
        )
        lcesn.initialize_reservoir_weights(
            strategies=res_weights['strategies'], spectral_scales=res_weights['spectral_scales'],
            offsets=res_weights['offsets']
        )
        lcesn.train(X_train, y_train)
        lcesn_outputs = []

        # generative tests
        u_n = data[split-1]
        for _ in range(len(valid_data)):
            u_n = lcesn.forward(u_n)
            lcesn_outputs.append(u_n)

        lcesn_outputs = np.array(lcesn_outputs).squeeze()

        error = nrmse(valid_data, lcesn_outputs, data_mean)
        print('NRMSE: %f\n' % error)
        results.append(error)
        
        if in_notebook:
            if run_num == n_runs - 1:
                progress_bar.close()
            else:
                progress_bar.value += 1

    total_time = time.time() - start_time
    print('Took %.3f seconds' % total_time)
    n_runs = len(results)
    key = [
        'echo_params: %f' % echo_params, 'regulariser: %f' % regulariser,
        'num_reservoirs: %d' % num_reservoirs, 'reservoir_sizes: %s' % reservoir_sizes,
        'in_weights: %s' % in_weights.items(), 'res_weights: %s' % res_weights.items()
    ]
    for i in range(len(key)-1):
        key[i] += '\n'
    key = ''.join(key)

    if key not in all_results.keys():
        all_results[key] = []

    all_results[key].extend(results)
    
    while 1:
        ch = raw_input('make sure you\'re not overriding old results. (y/n)')
        if ch == 'y':
            if fname is None:
                print('must provide a filename to save results')
                break
            elif fname[-2:] != '.p': 
                fname += '.p'
            
            class_str = str(esn_class)[16:-2]
            assert class_str in ['ESN', 'DHESN', 'LCESN', 'EESN']
            
            pkl.dump(all_results, open('Results/%s/%s.p' % (class_str, fname), 'wb'))
            break
        elif ch == 'n':
            print('okay, returning updated results dictionary instead')
            break
    
    return all_results
    
all_results = run_test(DHESN, all_results, fname='9_March.p')

IntProgress(value=0, max=150)

NRMSE: 461.799833

NRMSE: 1.144619

NRMSE: 1830.350125

NRMSE: 665117.382804

NRMSE: 245.667261

NRMSE: 2565.522192

NRMSE: 508.859625

NRMSE: 67.558291

NRMSE: 221.805672

NRMSE: 659.088762

NRMSE: 2.136218

NRMSE: 563.091449

NRMSE: 196.549513

NRMSE: 144.840508

NRMSE: 484.343840

NRMSE: 642886.829195

NRMSE: 666.374365

NRMSE: 227.898461

NRMSE: 1.783166

NRMSE: 167.264539

NRMSE: 170.450804

NRMSE: 5035.619183

NRMSE: 499.300633

NRMSE: 191.719742

NRMSE: 303.128191

NRMSE: 10571.073281

NRMSE: 668.961871

NRMSE: 177813.423972

NRMSE: 1.495895

NRMSE: 9463790.641718

NRMSE: 6983.519661

NRMSE: 2.131873



KeyboardInterrupt: 

# NRMSE histograms

In [ ]:
def show_nrmse_histograms(all_results, n_bins=30):
    for k, res in all_results.items():
        specs = k.split('\n')
        specs_dict = dict()
        for s in specs:
            #print(s)
            if 'reservoir_sizes' in s:
                sizes = s[s.index(':')+2:][1:-1]
                sizes = map(int, sizes.split(','))
            elif 'in_weights' in s or 'res_weights' in s:
                info = s[s.index(':')+2:]
                exec('info = dict(%s)' % info)
                k_ = 'in_weights' if 'in_weights' in s else 'res_weights'
                specs_dict[k_] = info
            else:
                k1 = "'%s'" % s[:s.index(':')]
                k2 = s[s.index(':') + 2:]
                exec('specs_dict[%s] = %s' % (k1, k2))

        # Check for infs, nans
        num_failed = 0
        res_clean = []
        for err in res:
            if np.isnan(err) or np.isinf(err) or err >= 1.:
                num_failed += 1
            else:
                res_clean.append(err)
        
        title = 'reg:%s. offset:%.1f. n_runs=%d. num_failures=%d' \
                    % (str(specs_dict['regulariser']), specs_dict['res_weights']['offsets'],
                       len(res), num_failed)
        if len(res_clean) > 0:
            hist, bins = np.histogram(res_clean, bins=min(len(res_clean), n_bins))
            bin_width = bins[1] - bins[0]

            f, ax = plt.subplots(figsize=(12, 6))
            ax.set_title(title)
            ax.bar(bins[:-1], hist, width=bin_width)
            plt.show()
        else:
            print('"%s" only yielded failures. oops' % title)
        raw_input()
        
#show_nrmse_histograms(all_results)

# State Histograms (ignore)